<a href="https://colab.research.google.com/github/janShi1105/science/blob/main/ML17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
! pip install -q tensorflow-gpu

     |████████████████████████████████| 578.0 MB 17 kB/s 
     |████████████████████████████████| 438 kB 71.3 MB/s 
     |████████████████████████████████| 1.7 MB 69.5 MB/s 
     |████████████████████████████████| 15.7 MB 53.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires keras<2.9,>=2.8.0rc0, but you have keras 2.10.0 which is incompatible.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.0 which is incompatible.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorflow-estimator<2.9,>=2.8, but you have tensorflow-estimator 2.10.0 which is incompatible.
plotnine 0.8.0 requires scipy>=1.5.0, but you have scipy 1.4.1 which is incompatible.
jaxlib 0.3.15+cuda11.cudnn805 requires scipy>=1.5, but you have scipy 1.4.1 which is incompatible.
jax 0.3.17 requ

In [25]:
import tensorflow as tf
print(tf.__version__)

print('GPU Avalilable: ', tf.config.list_physical_devices('GPU'))

if tf.config.list_physical_devices('GPU'):
  device_name = tf.test.gpu_device_name()
else: 
  device_name = '/CPU:0'
print(device_name)

2.8.2
GPU Avalilable:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
/device:GPU:0


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import tensorflow_datasets as tfds
import numpy as np

def make_generator_network(num_hidden_layers=1, num_hidden_units = 100, num_output_units =784):
  model = tf.keras.Sequential()
  for i in range(num_hidden_layers):
    model.add(tf.keras.layers.Dense(units=num_hidden_units, use_bias=False))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dense(units=num_output_units, activation='tanh'))
    return model

def make_discriminator_network(num_hidden_layers=1, num_hidden_units=100, num_output_units=1):
  model = tf.keras.Sequential()
  for i in range(num_hidden_layers):
    model.add(tf.keras.layers.Dense(units=num_hidden_units))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(rate=0.5))

  model.add(tf.keras.layers.Dense(units=num_output_units,activation=None))
  return model

In [9]:
image_size = (28,28)
z_size = 20
mode_z = 'uniform'
gen_hidden_layers = 1
gen_hidden_size = 100
disc_hidden_layers=1
disc_hidden_size = 100
tf.random.set_seed(1)
gen_model = make_generator_network(num_hidden_layers=gen_hidden_layers, num_hidden_units=gen_hidden_size, num_output_units=np.prod(image_size))
gen_model.build(input_shape=(None, z_size))
gen_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               2000      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 784)               79184     
                                                                 
Total params: 81,184
Trainable params: 81,184
Non-trainable params: 0
_________________________________________________________________


In [10]:
disc_model = make_discriminator_network(num_hidden_layers=disc_hidden_layers, num_hidden_units=disc_hidden_size)
disc_model.build(input_shape=(None, np.prod(image_size)))
disc_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 78,601
Trainable params: 78,601
Non-trainable params: 0
_________________________________________________________________


In [11]:
mnist_bldr = tfds.builder('mnist')
mnist_bldr.download_and_prepare()
mnist = mnist_bldr.as_dataset(shuffle_files=False)

Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to ~/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [12]:
def preprocess(ex,mode='uniform'):
  image = ex['image']
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.reshape(image, [-1])
  image = image*2-1.0
  if mode == 'uniform':
    input_z = tf.random.uniform(shape=(z_size,), minval=-1, maxval=1)
  elif mode == 'normal':
    input_z = tf.random.normal(shape=(z_size,))
  return input_z, image

In [14]:
mnist_trainset = mnist['train']
mnist_trainset = mnist_trainset.map(preprocess)

In [15]:
mnist_trainset = mnist_trainset.batch(32, drop_remainder=True)
input_z, input_real = next(iter(mnist_trainset))
print('input-z -- shape: ', input_z.shape)
print('input-real -- shape:',  input_real.shape)

input-z -- shape:  (32, 20)
input-real -- shape: (32, 784)


In [16]:
g_output = gen_model(input_z)
print('Output of G -- shape:', g_output.shape)

Output of G -- shape: (32, 784)


In [17]:
d_logits_real = disc_model(input_real)

In [18]:
d_logits_fake = disc_model(g_output)
print('Disc. (real) -- shape:', d_logits_real.shape)

Disc. (real) -- shape: (32, 1)


In [19]:
print('Disc. (fake) -- shape: ', d_logits_fake.shape)

Disc. (fake) -- shape:  (32, 1)


In [20]:
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
g_labels_real = tf.ones_like(d_logits_fake)
g_loss = loss_fn(y_true=g_labels_real, y_pred=d_logits_fake)
print('Generator Loss: {:.4f}'.format(g_loss))

Generator Loss: 0.7355


In [22]:
d_labels_real = tf.ones_like(d_logits_real)
d_labels_fake = tf.zeros_like(d_logits_fake)
d_loss_real = loss_fn(y_true=d_labels_real, y_pred=d_logits_real)
d_loss_fake = loss_fn(y_true=d_labels_fake, y_pred=d_labels_fake)
print('Discriminator Losses: Real {:.4f} Fake {:.4f}'.format(d_loss_real.numpy(), d_loss_fake.numpy()))

Discriminator Losses: Real 1.5073 Fake 0.6931


In [23]:
import time

num_epochs = 100
batch_size = 64
image_size = (28,28)
z_size = 20
mode_z = 'uniform'
gen_hidden_layers=1
gen_hidden_size = 100
disc_hidden_layers = 1
disc_hidden_size = 100

tf.random.set_seed(1)
np.random.seed(1)

if mode_z == 'uniform':
  fixed_z = tf.random.uniform(shape=(batch_size, z_size), minval=-1, maxval=1)
elif mode_z ==  'normal':
  fixed_z = tf.random.normal(shape=(batch_size, z_size))

def create_samples(g_model, input_z):
  g_output = g_model(input_z, training=False)
  images = tf.reshape(g_output, (batch_size, *image_size))
  return (images+1)/2

In [ ]:
mnist_trainset = mnist['train']
mnist_trainset = mnist_trainset.map(lambda ex: preprocess(ex, mode=mode_z))
mnist_trainset = mnist_trainset.shuffle(10000)
mnist_trainset = mnist_trainset.batch(batch_size, drop_remainder=True)

with tf.device(device_name):
  gen_model = make_generator_network(num_hidden_layers=gen_hidden_layers, num_hidden_units=gen_hidden_size,num_output_units=np.prod(image_size))
  gen_model.build(input_shape=(None, z_size))
  disc_model = make_discriminator_network(num_hidden_layers=disc_hidden_layers, num_hidden_units=disc_hidden_size)
  disc_model.build(input_shape=(None, np.prod(image_size)))

loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
g_optimizer = tf.keras.optimizers.Adam()
d_optimizer = tf.keras.optimizers.Adam()

all_losses = []
all_d_vals = []
epoch_samples = []

start_time = time.time()
for epoch in range(1, num_epochs+1):
  epoch_losses, epoch_d_vals = [], []
  for i , (input_z, input_real) in enumerate(mnist_trainset):
    with tf.GradientTape() as g_tape:
      g_output = gen_model(input_z)
      d_logits_fake = disc_model(g_output, training=True)
      labels_real = tf.ones_like(d_logits_fake)
      g_loss = loss_fn(y_true=labels_real, y_pred=d_logits_fake)

    g_grads = g_tape.gradient(g_loss, gen_model.trainable_variables)
    g_optimizer.apply_gradients(grads_and_vars=zip(g_grads, gen_model.trainable_variables))

    with tf.GradientTape() as d_tape:
      d_logits_real = disc_model(input_real, training=True)
      d_labels_real = tf.ones_like(d_logits_real)
      d_loss_real = loss_fn(y_true=d_labels_real, y_pred=d_logits_real)
      d_logits_fake = disc_model(g_output, training=True)
      d_labels_fake = tf.zeros_like(d_logits_fake)
      d_loss_fake = loss_fn(y_true=d_labels_fake, y_pred=d_logits_fake)
      d_loss = d_loss_real + d_loss_fake

    d_grads = d_tape.gradient(d_loss, disc_model.trainable_variables)
    d_optimizer.apply_gradients(grads_and_vars=zip(d_grads, disc_model.trainable_variables))
    epoch_losses.append((g_loss.numpy(), d_loss.numpy(), d_loss_real.numpy(), d_loss_fake.numpy()))
    d_probs_real = tf.reduce_mean(tf.sigmoid(d_logits_real))
    d_probs_fake = tf.reduce_mean(tf.sigmoid(d_logits_fake))
    epoch_d_vals.append((d_probs_real.numpy(), d_probs_fake.numpy()))

  all_losses.apped(epoch_losses)
  all_d_vals.append(epoch_d_vals)
  print('Epoch {:03d} | ET {:.2f} min | Avg Losses')